In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
#from plotly.offline import init_notebook_mode, iplot
#import plotly.graph_objs as go
import os
import warnings

# 파일 불러와서 열이름 바꾸기
korean, long_term_frgn, short_term_frgn, resident, card_cnt, card_amt, waste_cnt
(이거 예시는 waste_cnt임)

In [4]:
##### 파일 불러오기 #####
df = pd.read_csv("C:\\Users\\asus\\BigContest\\code\\unknown_monthly.csv", encoding='cp949')
df = df.drop(['em_cnt'], 1)
df

,em_g
2018-01-31,8880900
2018-02-28,7543950
2018-03-31,9287150
2018-04-30,9853350
2018-05-31,10290950
2018-06-30,10533050
2018-07-31,11860200
2018-08-31,12184350
2018-09-30,9085850
2018-10-31,9371850


In [5]:
sub_df = df.copy()

In [6]:
# series 형으로 변환
sub_df.to_dict('series')

{'em_g': 2018-01-31     8880900
 2018-02-28     7543950
 2018-03-31     9287150
 2018-04-30     9853350
 2018-05-31    10290950
 2018-06-30    10533050
 2018-07-31    11860200
 2018-08-31    12184350
 2018-09-30     9085850
 2018-10-31     9371850
 2018-11-30     9997100
 2018-12-31     8593600
 2019-01-31     5044050
 2019-02-28           0
 2019-03-31           0
 2019-04-30           0
 2019-05-31           0
 2019-06-30           0
 2019-07-31           0
 2019-08-31           0
 2019-09-30      482950
 2019-10-31      550350
 2019-11-30      621300
 2019-12-31      828350
 2020-01-31      645900
 2020-02-29      337600
 2020-03-31           0
 2020-04-30           0
 2020-05-31           0
 2020-06-30      836200
 2020-07-31     1372799
 2020-08-31     2387650
 2020-09-30     2010300
 2020-10-31     2226900
 2020-11-30     2088750
 2020-12-31     2147470
 2021-01-31     2057700
 2021-02-28     1985100
 2021-03-31     2153800
 2021-04-30     1936900
 2021-05-31     1910750
 2021-06

# ARIMA

In [7]:
# ARIMA에 적합한 log로 바꾸는 과정
sub_df_log = np.log1p(sub_df[:])
#sub_df_float.tail()

In [8]:
# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
from statsmodels.tsa.stattools import adfuller

result = adfuller(sub_df_log)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))
    
##### 여기서 p-value 확인 -> 0.05보다 크면 아래로 내려가서 차분하기 / 작으면 차분 생략(아래 셀 생략) #####

ADF Statistic : -2.243308
p-value : 0.190904
Critical Values :
	1%: -3.600983
	5%: -2.935135
	10%: -2.605963


In [9]:
# 차분
diff_1 = sub_df_log.diff(periods=1).iloc[1:]

# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
result = adfuller(diff_1)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))
    
##### 여기서 또 p-value 확인 -> 0.05보다 크면 얼른 단톡에 보고하기 / 작으면 아래로 쭉 진행 #####

ADF Statistic : -5.962801
p-value : 0.000000
Critical Values :
	1%: -3.605565
	5%: -2.937069
	10%: -2.606986


In [10]:
##### 이건 설치하는 거라 처음 돌릴 때 한 번만 주석 풀어서 설치하고 그 이후로는 필요없음! #####

#!pip install pmdarima

In [11]:
from pmdarima.arima import auto_arima

model_arima= auto_arima(sub_df_log,trace=True, error_action='ignore',suppress_warnings=True,stepwise=False,seasonal=True)

model_arima.fit(sub_df_log)

##### 여기서 아래쪽에 'Best model:  ARIMA(1,1,1)(0,0,0)[1] intercept' 이렇게 나오면 (1,1,1) 가져다 쓸거임 #####
##### ARIMA(1,1,1) 여기 숫자 세 개만 잘 보면 됨 !!!! #####

 ARIMA(0,0,0)(0,0,0)[1] intercept   : AIC=279.164, Time=0.16 sec
 ARIMA(0,0,1)(0,0,0)[1] intercept   : AIC=257.612, Time=0.11 sec
 ARIMA(0,0,2)(0,0,0)[1] intercept   : AIC=245.973, Time=0.11 sec
 ARIMA(0,0,3)(0,0,0)[1] intercept   : AIC=243.202, Time=0.13 sec
 ARIMA(0,0,4)(0,0,0)[1] intercept   : AIC=245.109, Time=0.30 sec
 ARIMA(0,0,5)(0,0,0)[1] intercept   : AIC=247.060, Time=0.25 sec
 ARIMA(1,0,0)(0,0,0)[1] intercept   : AIC=243.076, Time=0.05 sec
 ARIMA(1,0,1)(0,0,0)[1] intercept   : AIC=244.364, Time=0.08 sec
 ARIMA(1,0,2)(0,0,0)[1] intercept   : AIC=243.741, Time=0.16 sec
 ARIMA(1,0,3)(0,0,0)[1] intercept   : AIC=245.725, Time=0.18 sec
 ARIMA(1,0,4)(0,0,0)[1] intercept   : AIC=247.171, Time=0.36 sec
 ARIMA(2,0,0)(0,0,0)[1] intercept   : AIC=244.009, Time=0.10 sec
 ARIMA(2,0,1)(0,0,0)[1] intercept   : AIC=245.171, Time=0.22 sec
 ARIMA(2,0,2)(0,0,0)[1] intercept   : AIC=245.739, Time=0.23 sec
 ARIMA(2,0,3)(0,0,0)[1] intercept   : AIC=247.690, Time=0.38 sec
 ARIMA(3,0,0)(0,0,0)[1] i

ARIMA(order=(1, 0, 0), scoring_args={}, seasonal_order=(0, 0, 0, 1),
      suppress_warnings=True)

In [13]:
# ARIMA 모델 
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

# (AR=2, 차분=1, MA=2) 파라미터로 ARIMA 모델을 학습합니다.
model = ARIMA(sub_df_log.em_g.values, order=(1,0,0))     ##### 여기 order=(1,1,1)에 위에서 나온 숫자 넣는거 #####
                                   ##### 그리고 sub_df_log.waste_cnt.values에서 가운데 waste_cnt 각자걸로 바꾸기 #####

#trend : constant를 가지고 있는지, c - constant / nc - no constant
#disp : 수렴 정보를 나타냄
model_fit = model.fit(trend='c', full_output=True, disp=True)
print(model_fit.summary())

                              ARMA Model Results                              
Dep. Variable:                      y   No. Observations:                   42
Model:                     ARMA(1, 0)   Log Likelihood                -118.538
Method:                       css-mle   S.D. of innovations              4.026
Date:                Sat, 11 Sep 2021   AIC                            243.076
Time:                        19:22:32   BIC                            248.289
Sample:                             0   HQIC                           244.987
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         11.8656      2.495      4.755      0.000       6.975      16.757
ar.L1.y        0.7674      0.094      8.180      0.000       0.584       0.951
                                    Roots           

C:\Users\asus\anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


# 예측하기

In [14]:
# 2단위 이후의 예측결과
fore = model_fit.forecast(steps=2)
print(fore)

##### 맨 앞에 나온 'array([10681271.81014634, 10741037.02029268])' 이게 차례대로 7, 8월 예측값에 로그씌운거임 #####
##### 7월 : 10.05681231, 8월 : 10.28080669 #####
##### 이거 지수함수에 넣어서 원래값으로 돌려야 함 -> 각각 가져다 밑에 넣기 #####

(array([13.95230321, 13.46703176]), array([4.02614459, 5.07513538]), array([[ 6.06120482, 21.84340161],
       [ 3.51994919, 23.41411433]]))


In [16]:
# 이거 위에서 로그씌울 때 np.log1p 이거 써서 지수함수에 넣은 다음에 1 뺀거임
# https://rfriend.tistory.com/295
# 궁금하면 이 링크 들어가서 읽어보세요 질문받기 싫어서 미리 써놓음
# 밑에 어떻게 되는지 예제코드도 넣었으니까 제발 질문 안해주길 ㅎ

print( np.exp(13.95230321) - 1 )   ##### 괄호 안에 7월 숫자 넣기 #####
print( np.exp(13.46703176) - 1 )   ##### 괄호 안에 8월 숫자 넣기 #####

##### 이 값 가져다 엑셀파일에 입력하기 #####

1146589.3806896203
705759.8784326583


In [138]:
a = 134
lna = np.log1p(a)  # a에 자연로그씌운거
ea = np.exp(lna)  # 다시 지수함수에 넣은거
print(a, lna, ea)  # 결과가 1 크게 나옴

134 4.90527477843843 135.00000000000003
